In [518]:
import pandas as pd

In [519]:
#setup the dataframe that includes rows from the whole datast
hitting_df_original = pd.read_csv('Batting_final_project.csv')

In [520]:
#filter only the rows that are between the years 1984 and 2014
hitting_df_filtered = hitting_df_original[(hitting_df_original['yearID'] >= 1984) & (hitting_df_original['yearID'] <= 2014)]

In [521]:
#display the new dataframe
hitting_df_filtered.head()

playerID  yearID  stint teamID lgID   G  G_batting  AB  R  H  ...   SB  \
0  aardsda01    2004      1    SFN   NL  11        NaN   0  0  0  ...  0.0   
1  aardsda01    2006      1    CHN   NL  45        NaN   2  0  0  ...  0.0   
2  aardsda01    2007      1    CHA   AL  25        NaN   0  0  0  ...  0.0   
3  aardsda01    2008      1    BOS   AL  47        NaN   1  0  0  ...  0.0   
4  aardsda01    2009      1    SEA   AL  73        NaN   0  0  0  ...  0.0   

    CS  BB   SO  IBB  HBP   SH   SF  GIDP  G_old  
0  0.0   0  0.0  0.0  0.0  0.0  0.0   0.0    NaN  
1  0.0   0  0.0  0.0  0.0  1.0  0.0   0.0    NaN  
2  0.0   0  0.0  0.0  0.0  0.0  0.0   0.0    NaN  
3  0.0   0  1.0  0.0  0.0  0.0  0.0   0.0    NaN  
4  0.0   0  0.0  0.0  0.0  0.0  0.0   0.0    NaN  

[5 rows x 24 columns]

In [522]:
#drop hitting dataframe columns
hitting_df_filtered= hitting_df_filtered.drop(columns = ['G', 'SO', 'RBI', 'stint', 'teamID', 'lgID', 'G_batting', 'SB', 'CS', 'IBB', 'SH', 'SF', 'GIDP', 'G_old'])

In [523]:
#drop rows with a '0' in all of the columns
hitting_df_cleaned = hitting_df_filtered[(hitting_df_filtered != 0).all(axis=1)]

In [524]:
print('hitting_df_cleaned')
hitting_df_cleaned.head()

hitting_df_cleaned


playerID  yearID   AB   R    H  2B  3B  HR  BB  HBP
113  abbotje01    2000  215  31   59  15   1   3  21  2.0
127  abbotku01    1994  345  41   86  17   3   9  16  5.0
128  abbotku01    1995  420  60  107  18   7  17  36  5.0
129  abbotku01    1996  320  37   81  18   7   8  22  3.0
130  abbotku01    1997  252  35   69  18   2   6  14  1.0

In [525]:
#drop players that have less than 222 'AB's, the lower quartile range cutoff
hitting_df_cleaned = hitting_df_cleaned[hitting_df_cleaned['AB'] >= 222]

In [526]:
#drop players that have less than 3 Home Runs, the lower quartile range cutoff
hitting_df_cleaned = hitting_df_cleaned[hitting_df_cleaned['HR'] >= 3]

In [527]:
# Sort data by playerID and yearID
hitting_df_cleaned = hitting_df_cleaned.sort_values(by=['playerID', 'yearID'])

In [528]:
hitting_df_cleaned.head(5)

playerID  yearID   AB   R    H  2B  3B  HR  BB  HBP
127  abbotku01    1994  345  41   86  17   3   9  16  5.0
128  abbotku01    1995  420  60  107  18   7  17  36  5.0
129  abbotku01    1996  320  37   81  18   7   8  22  3.0
130  abbotku01    1997  252  35   69  18   2   6  14  1.0
162  abercre01    2006  255  39   54  12   2   5  18  3.0

In [529]:
# Display summary statistics of filtered hitting dataframe
hitting_df_cleaned.describe().round(3)

yearID        AB         R         H        2B        3B        HR  \
count  5769.000  5769.000  5769.000  5769.000  5769.000  5769.000  5769.000   
mean   1999.839   455.809    66.156   125.561    24.633     3.179    15.237   
std       8.683   117.953    24.492    38.887     9.399     2.460     9.923   
min    1984.000   222.000    11.000    41.000     4.000     1.000     3.000   
25%    1993.000   359.000    46.000    94.000    17.000     1.000     8.000   
50%    2000.000   469.000    64.000   126.000    24.000     2.000    13.000   
75%    2007.000   557.000    84.000   156.000    31.000     4.000    21.000   
max    2014.000   716.000   152.000   262.000    59.000    23.000    73.000   

             BB       HBP  
count  5769.000  5769.000  
mean     45.881     4.524  
std      22.950     3.723  
min       4.000     1.000  
25%      29.000     2.000  
50%      42.000     4.000  
75%      58.000     6.000  
max     232.000    35.000

# Feature Engineering

A few columns need to be added to get a good idea of how players performances were compared to previous years. We will add Batting Average (BA), Slugging Percentage (SLG), On-Base Percentage (OBP), and On-Base Plust Slugging Percentage (OPS).

#### Batting Average (BA)

The batting Average (BA) is the measure of how often a player gets a hit, no matter what type of hit it is. 
    
The formula for batting average is:

$$ \text{BA} = \frac{\text{H}}{\text{AB}} $$


In [530]:
# Calculate Batting Average and add it to the cleaned dataframe
hitting_df_cleaned['BA'] = hitting_df_cleaned['H'] / hitting_df_cleaned['AB']

#### Slugging Percentage

The Slugging Percentage (SLG) in baseball is a measure of the power of a hitter. It is calculated as the total number of bases a player records per at-bat. 

The formula for slugging percentage is:

$$ \text{SLG} = \frac{\text{1B} + (2 \times \text{2B}) + (3 \times \text{3B}) + (4 \times \text{HR})}{\text{AB}} $$

The total bases are calculated as:

$$ \text{Total Bases} = \text{1B} + (2 \times \text{2B}) + (3 \times \text{3B}) + (4 \times \text{HR}) $$

The number of Singles is calculated as:

$$ \text{1B} = \text{H} - \text{2B} - \text{3B} - \text{HR} $$

In [531]:
# Calculate Slugger Percentage (SLG) and add it to the cleaned dataframe
hitting_df_cleaned['SLG'] = ((hitting_df_cleaned['H'] - hitting_df_cleaned['2B'] - hitting_df_cleaned['3B'] - hitting_df_cleaned['HR']) 
    + (2 * hitting_df_cleaned['2B']) + (3 * hitting_df_cleaned['3B']) + (4 * hitting_df_cleaned['HR'])) / hitting_df_cleaned['AB']

#### On-Base Percentage

The On-Base Percentage (OBP) in baseball measures how frequently a batter reaches base per plate appearance. This includes Hit-By Pitches and Walks. The formula for on-base percentage is:

$$ \text{OBP} = \frac{\text{H} + \text{BB} + \text{HBP}}{\text{AB} + \text{BB} + \text{HBP}} $$

In [532]:
# Calculate On-Base Percentage
hitting_df_cleaned['OBP'] = (hitting_df_cleaned['H'] + hitting_df_cleaned['BB'] + hitting_df_cleaned['HBP']) / (hitting_df_cleaned['AB'] 
    + hitting_df_cleaned['BB'] + hitting_df_cleaned['HBP'])

#### On-Base Plus Slugging
OPS (On-base Plus Slugging) is a popular statistic in baseball that combines a player's on-base percentage (OBP) and slugging percentage (SLG) to give a single measure of a player's offensive performance.

You can calculate OPS by simply adding the OBP and SLG values for each player.

The formula for on-base plus sugging percentage is:

$$ \text{OPS} = \text{OBP} + \text{SLG} $$

In [533]:
# Calculate OPS (On-base Plus Slugging) and add it to the cleaned dataframe
hitting_df_cleaned['OPS'] = hitting_df_cleaned['OBP'] + hitting_df_cleaned['SLG']

In [534]:
# Add a column for the number of home runs from the previous year
hitting_df_cleaned['HR_prior'] = hitting_df_cleaned.groupby('playerID')['HR'].shift(1)

#### Summary Statistics

In [535]:
# Display summary statistics of filtered hitting dataframe
hitting_df_cleaned.describe().round(3)

yearID        AB         R         H        2B        3B        HR  \
count  5769.000  5769.000  5769.000  5769.000  5769.000  5769.000  5769.000   
mean   1999.839   455.809    66.156   125.561    24.633     3.179    15.237   
std       8.683   117.953    24.492    38.887     9.399     2.460     9.923   
min    1984.000   222.000    11.000    41.000     4.000     1.000     3.000   
25%    1993.000   359.000    46.000    94.000    17.000     1.000     8.000   
50%    2000.000   469.000    64.000   126.000    24.000     2.000    13.000   
75%    2007.000   557.000    84.000   156.000    31.000     4.000    21.000   
max    2014.000   716.000   152.000   262.000    59.000    23.000    73.000   

             BB       HBP        BA       SLG       OBP       OPS  HR_prior  
count  5769.000  5769.000  5769.000  5769.000  5769.000  5769.000  4390.000  
mean     45.881     4.524     0.273     0.439     0.344     0.783    16.547  
std      22.950     3.723     0.030     0.070     0.038     0.099    10.213  
min       4.000     1.000     0.163     0.262     0.230     0.527     3.000  
25%      29.000     2.000     0.252     0.388     0.318     0.714     9.000  
50%      42.000     4.000     0.272     0.432     0.342     0.774    14.000  
75%      58.000     6.000     0.292     0.481     0.367     0.841    23.000  
max     232.000    35.000     0.394     0.863     0.612     1.425    73.000

Now that the percentages are calculate, we can clean up the dataframe and remove some of the features we used to calculate percentage since we no longer need them.

In [536]:
# Drop hitting dataframe columns we don't need anymore.
hitting_df_cleaned= hitting_df_cleaned.drop(columns = ['2B', '3B', 'BB', 'R', 'HBP', 'OBP'])

In [537]:
hitting_df_cleaned.head()

playerID  yearID   AB    H  HR        BA       SLG       OPS  HR_prior
127  abbotku01    1994  345   86   9  0.249275  0.394203  0.686553       NaN
128  abbotku01    1995  420  107  17  0.254762  0.452381  0.773422       9.0
129  abbotku01    1996  320   81   8  0.253125  0.428125  0.735371      17.0
130  abbotku01    1997  252   69   6  0.273810  0.432540  0.747146       8.0
162  abercre01    2006  255   54   5  0.211765  0.333333  0.605072       NaN

#### Percent Increase from previous year

In [538]:
# Calculate percent increase in HR for each player
hitting_df_cleaned['HR_increase'] = hitting_df_cleaned.groupby('playerID')['HR'].pct_change() * 100

In [539]:
# Calculate percent increase in Batting Average for each player
hitting_df_cleaned['BA_increase'] = hitting_df_cleaned.groupby('playerID')['BA'].pct_change() * 100

In [540]:
# Calculate percent increase in Slugging Percentage for each player
hitting_df_cleaned['SLG_increase'] = hitting_df_cleaned.groupby('playerID')['SLG'].pct_change() * 100

In [541]:
# Calculate percent increase in On-Base Plus Slugging for each player
hitting_df_cleaned['OPS_increase'] = hitting_df_cleaned.groupby('playerID')['OPS'].pct_change() * 100

In [542]:
# Replace NaNs with 0s
hitting_df_cleaned = hitting_df_cleaned.fillna(0)

To keep the data as anonymous as possible, we can now drop Player ID from the dataframe.t

In [543]:
# Drop 'playerID' and 'OBP' from dataframe
hitting_df_cleaned= hitting_df_cleaned.drop(columns = ['playerID'])

In [544]:
hitting_df_cleaned = hitting_df_cleaned.round(3)

In [545]:
hitting_df_cleaned.head()

yearID   AB    H  HR     BA    SLG    OPS  HR_prior  HR_increase  \
127    1994  345   86   9  0.249  0.394  0.687       0.0        0.000   
128    1995  420  107  17  0.255  0.452  0.773       9.0       88.889   
129    1996  320   81   8  0.253  0.428  0.735      17.0      -52.941   
130    1997  252   69   6  0.274  0.433  0.747       8.0      -25.000   
162    2006  255   54   5  0.212  0.333  0.605       0.0        0.000   

     BA_increase  SLG_increase  OPS_increase  
127        0.000         0.000         0.000  
128        2.201        14.758        12.653  
129       -0.643        -5.362        -4.920  
130        8.172         1.031         1.601  
162        0.000         0.000         0.000

In [546]:
print('Correlations')
hitting_df_cleaned.corr().round(2)

Correlations


yearID    AB     H    HR    BA   SLG   OPS  HR_prior  \
yearID          1.00 -0.01 -0.01  0.04 -0.02  0.05  0.03      0.11   
AB             -0.01  1.00  0.94  0.49  0.34  0.27  0.29      0.30   
H              -0.01  0.94  1.00  0.50  0.62  0.44  0.49      0.29   
HR              0.04  0.49  0.50  1.00  0.27  0.82  0.73      0.59   
BA             -0.02  0.34  0.62  0.27  1.00  0.64  0.74      0.13   
SLG             0.05  0.27  0.44  0.82  0.64  1.00  0.96      0.46   
OPS             0.03  0.29  0.49  0.73  0.74  0.96  1.00      0.44   
HR_prior        0.11  0.30  0.29  0.59  0.13  0.46  0.44      1.00   
HR_increase    -0.00  0.22  0.24  0.28  0.16  0.26  0.23     -0.21   
BA_increase    -0.04  0.07  0.22  0.12  0.49  0.34  0.37     -0.11   
SLG_increase   -0.05  0.06  0.18  0.25  0.39  0.44  0.42     -0.21   
OPS_increase   -0.06  0.06  0.20  0.23  0.43  0.43  0.44     -0.19   

              HR_increase  BA_increase  SLG_increase  OPS_increase  
yearID              -0.00        -0.04         -0.05         -0.06  
AB                   0.22         0.07          0.06          0.06  
H                    0.24         0.22          0.18          0.20  
HR                   0.28         0.12          0.25          0.23  
BA                   0.16         0.49          0.39          0.43  
SLG                  0.26         0.34          0.44          0.43  
OPS                  0.23         0.37          0.42          0.44  
HR_prior            -0.21        -0.11         -0.21         -0.19  
HR_increase          1.00         0.29          0.62          0.53  
BA_increase          0.29         1.00          0.79          0.87  
SLG_increase         0.62         0.79          1.00          0.96  
OPS_increase         0.53         0.87          0.96          1.00

There are a few strong correlations we can look at.

AB and H: 0.9447060.944706

HR and SLG: 0.8165460.816546

SLG and OPS: 0.9594220.959422

OPS_increase and SLG_increase: 0.9649740.964974

OPS_increase and BA_increase: 0.8660320.866032

In [547]:
# Save the dataframe as a .csv file
hitting_df_cleaned.to_csv('hitting_df_cleaned.csv', index=False)